<a href="https://colab.research.google.com/github/MaidenTaief/Music_Final/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install transformers and import necessary libraries
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00


In [3]:
import pandas as pd

# Load the dataset
dataset_path = '/content/drive/My Drive/DATASET/aggregated_data_full.csv'
df = pd.read_csv(dataset_path)

# Extract unique genres
unique_genres = set()
df['genres'].str.lower().str.split(' / ').apply(lambda x: unique_genres.update(x) if isinstance(x, list) else None)
unique_genres = list(unique_genres)

# Extract unique cluster descriptions
unique_clusters = df['Cluster_Description'].unique().tolist()

# Save unique genres and clusters to a text file
with open('/content/drive/My Drive/DATASET/unique_genres_clusters.txt', 'w') as f:
    f.write("Unique Genres:\n")
    f.writelines(f"{genre}\n" for genre in unique_genres)
    f.write("\nUnique Cluster Descriptions:\n")
    f.writelines(f"{cluster}\n" for cluster in unique_clusters)

print("Data saved to 'unique_genres_clusters.txt'")

Data saved to 'unique_genres_clusters.txt'


In [4]:
print(df.head())
print(df.sample(5))
print(df[['trackname', 'artistname', 'genres', 'genre1', 'genre2', 'decade', 'normalized_popularity']].head())
print(df.describe())
# Summing each genre column to see how many songs are classified under each genre
genre_columns = [genre for genre in all_genres]  # List of all genre column names you created
print(df[genre_columns].sum().sort_values(ascending=False))
print(df.isnull().sum())

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

# Enhanced mapping dictionary
genre_keywords = {
    'rock': ['rock', 'punk', 'grunge', 'alternative', 'indie rock', 'garage rock', 'psychedelic rock', 'classic rock', 'folk rock', 'hard rock', 'prog rock', 'post-rock'],
    'pop': ['pop', 'synthpop', 'indie pop', 'k-pop', 'pop rock', 'electropop', 'bubblegum pop', 'dance pop', 'pop punk'],
    'hip hop': ['hip hop', 'rap', 'trap', 'gangster rap', 'old school hip hop', 'underground hip hop', 'latin hip hop', 'jazz rap'],
    'electronic': ['electronic', 'edm', 'house', 'techno', 'dubstep', 'trance', 'drum and bass', 'ambient', 'electro', 'industrial', 'techno'],
    'jazz': ['jazz', 'bebop', 'fusion', 'swing', 'smooth jazz', 'jazz funk', 'jazz fusion', 'acid jazz', 'jazz blues'],
    'classical': ['classical', 'opera', 'symphony', 'orchestral', 'baroque', 'chamber music', 'sonata', 'classical performance', 'early music'],
    'folk': ['folk', 'acoustic', 'singer-songwriter', 'folktronica', 'celtic', 'americana', 'bluegrass', 'folk rock'],
    'blues': ['blues', 'soul', 'gospel', 'r&b', 'soul blues', 'southern soul', 'blues rock', 'chicago blues', 'delta blues'],
    'reggae': ['reggae', 'dancehall', 'ska', 'dub', 'reggaeton', 'rocksteady', 'ragga', 'reggae fusion'],
    'country': ['country', 'country rock', 'alt-country', 'country blues', 'bluegrass', 'honky tonk', 'country pop', 'country soul'],
    'latin': ['latin', 'salsa', 'tango', 'reggaeton', 'latin pop', 'latin jazz', 'bossa nova', 'merengue', 'bachata', 'cumbia'],
    'world': ['world', 'afrobeat', 'bollywood', 'fado', 'flamenco', 'k-pop', 'j-pop', 'regional mexican', 'samba', 'world fusion'],
    'electronic variations': ['ambient', 'synthwave', 'vaporwave', 'chillwave', 'downtempo', 'lo-fi', 'future bass', 'trip hop'],
    'metal variations': ['metal', 'death metal', 'black metal', 'doom metal', 'thrash metal', 'folk metal', 'gothic metal', 'metalcore', 'viking metal'],
    'regional': ['french pop', 'italian pop', 'german pop', 'spanish pop', 'swedish pop', 'russian pop', 'polish pop', 'balkan', 'turkish pop']

}

mood_context_mapping = {
    'chill': ['Mellow / Vibrant / Joyful', 'Instrument-rich / Mellow / Danceable'],
    'vibrant': ['Varied Tempo / Vibrant / Mellow', 'Instrument-rich / Mellow / Danceable'],
    'joyful': ['Joyful / Danceable / Live', 'Mellow / Vibrant / Joyful'],
    'danceable': ['Joyful / Danceable / Live', 'Lyrical / Live / Danceable'],
    'lyrical': ['Lyrical / Live / Danceable'],
    'sad': ['Mellow / Vibrant / Joyful'],  # Assuming sad music might be more mellow and introspective
    'energetic': ['Joyful / Danceable / Live'],  # High-energy music for workouts or lively activities
    'relaxing': ['Instrument-rich / Mellow / Danceable', 'Mellow / Vibrant / Joyful'],  # Music for relaxation or stress relief
    'romantic': ['Mellow / Vibrant / Joyful'],  # Music that sets a romantic mood
    'intense': ['Varied Tempo / Vibrant / Mellow'],  # For thrilling or emotionally intense music
    'meditative': ['Instrument-rich / Mellow / Danceable']  # Music conducive to meditation or deep thought
}

# Convert genre keywords dictionary to DataFrame
genre_df = pd.DataFrame(list(genre_keywords.items()), columns=['Genre', 'Keywords'])
genre_df = genre_df.explode('Keywords')  # Expand lists to individual rows

# Convert mood context dictionary to DataFrame
mood_df = pd.DataFrame(list(mood_context_mapping.items()), columns=['Mood', 'Clusters'])
mood_df = mood_df.explode('Clusters')  # Expand lists to individual rows



In [8]:
import pandas as pd
import re

# Enhanced mapping dictionary
genre_keywords = {
    'rock': ['rock', 'punk', 'grunge', 'alternative', 'indie rock', 'garage rock', 'psychedelic rock', 'classic rock', 'folk rock', 'hard rock', 'prog rock', 'post-rock'],
    'pop': ['pop', 'synthpop', 'indie pop', 'k-pop', 'pop rock', 'electropop', 'bubblegum pop', 'dance pop', 'pop punk'],
    'hip hop': ['hip hop', 'rap', 'trap', 'gangster rap', 'old school hip hop', 'underground hip hop', 'latin hip hop', 'jazz rap'],
    'electronic': ['electronic', 'edm', 'house', 'techno', 'dubstep', 'trance', 'drum and bass', 'ambient', 'electro', 'industrial', 'techno'],
    'jazz': ['jazz', 'bebop', 'fusion', 'swing', 'smooth jazz', 'jazz funk', 'jazz fusion', 'acid jazz', 'jazz blues'],
    'classical': ['classical', 'opera', 'symphony', 'orchestral', 'baroque', 'chamber music', 'sonata', 'classical performance', 'early music'],
    'folk': ['folk', 'acoustic', 'singer-songwriter', 'folktronica', 'celtic', 'americana', 'bluegrass', 'folk rock'],
    'blues': ['blues', 'soul', 'gospel', 'r&b', 'soul blues', 'southern soul', 'blues rock', 'chicago blues', 'delta blues'],
    'reggae': ['reggae', 'dancehall', 'ska', 'dub', 'reggaeton', 'rocksteady', 'ragga', 'reggae fusion'],
    'country': ['country', 'country rock', 'alt-country', 'country blues', 'bluegrass', 'honky tonk', 'country pop', 'country soul'],
    'latin': ['latin', 'salsa', 'tango', 'reggaeton', 'latin pop', 'latin jazz', 'bossa nova', 'merengue', 'bachata', 'cumbia'],
    'world': ['world', 'afrobeat', 'bollywood', 'fado', 'flamenco', 'k-pop', 'j-pop', 'regional mexican', 'samba', 'world fusion'],
    'electronic variations': ['ambient', 'synthwave', 'vaporwave', 'chillwave', 'downtempo', 'lo-fi', 'future bass', 'trip hop'],
    'metal variations': ['metal', 'death metal', 'black metal', 'doom metal', 'thrash metal', 'folk metal', 'gothic metal', 'metalcore', 'viking metal'],
    'regional': ['french pop', 'italian pop', 'german pop', 'spanish pop', 'swedish pop', 'russian pop', 'polish pop', 'balkan', 'turkish pop']

}

mood_context_mapping = {
    'chill': ['Mellow / Vibrant / Joyful', 'Instrument-rich / Mellow / Danceable'],
    'vibrant': ['Varied Tempo / Vibrant / Mellow', 'Instrument-rich / Mellow / Danceable'],
    'joyful': ['Joyful / Danceable / Live', 'Mellow / Vibrant / Joyful'],
    'danceable': ['Joyful / Danceable / Live', 'Lyrical / Live / Danceable'],
    'lyrical': ['Lyrical / Live / Danceable'],
    'sad': ['Mellow / Vibrant / Joyful'],  # Assuming sad music might be more mellow and introspective
    'energetic': ['Joyful / Danceable / Live'],  # High-energy music for workouts or lively activities
    'relaxing': ['Instrument-rich / Mellow / Danceable', 'Mellow / Vibrant / Joyful'],  # Music for relaxation or stress relief
    'romantic': ['Mellow / Vibrant / Joyful'],  # Music that sets a romantic mood
    'intense': ['Varied Tempo / Vibrant / Mellow'],  # For thrilling or emotionally intense music
    'meditative': ['Instrument-rich / Mellow / Danceable']  # Music conducive to meditation or deep thought
}

# Define a function to generate keywords for decades
def generate_decade_keywords():
    temporal_keywords = {}
    # Cover each decade from the 1920s to the 2020s
    for year in range(1920, 2030, 10):
        decade = year // 10 * 10
        decade_key = f"{decade}s"
        temporal_keywords[decade_key] = [
            str(decade),  # "1920"
            f"{str(decade)[2:]}s",  # "20s"
            f"{decade}s"  # "1920s"
        ]
    return temporal_keywords

# Generate the dictionary with temporal keywords
temporal_keywords = generate_decade_keywords()

# Extend the genre_keywords dictionary with newly generated temporal mappings
def map_query_to_genres_and_moods(query):
    query = query.lower()
    found_genres = set()
    found_moods = set()
    for genre, keywords in genre_keywords.items():  # Ensure genre_keywords is defined and filled
        if any(keyword in query for keyword in keywords):
            found_genres.add(genre)
    for mood, clusters in mood_context_mapping.items():
        if any(mood in query for cluster in clusters):
            found_moods.update(clusters)
    return found_genres, found_moods

def map_query_to_decades(query, temporal_keywords):
    query = query.lower()
    found_decades = {}
    for decade, keywords in temporal_keywords.items():
        if any(keyword in query for keyword in keywords):
            found_decades[decade] = keywords
    return found_decades

def map_query_to_genres_moods_and_decades(query):
    genres, moods = map_query_to_genres_and_moods(query)
    decades = map_query_to_decades(query, temporal_keywords)
    return genres, moods, decades

# Test the new setup with a few queries:
test_queries = [
    "I want to listen to some chill jazz music from the 60s",
    "Find me sad rock songs from the 1980s",
    "Can you play some romantic classic jazz from the 20s?"
]

for query in test_queries:
    genres, moods, decades = map_query_to_genres_moods_and_decades(query)
    print(f"Query: {query}")
    print(f"Mapped Genres: {genres}")
    print(f"Mapped Moods: {moods}")
    print(f"Mapped Decades: {decades}\n")


Query: I want to listen to some chill jazz music from the 60s
Mapped Genres: {'jazz'}
Mapped Moods: {'Instrument-rich / Mellow / Danceable', 'Mellow / Vibrant / Joyful'}
Mapped Decades: {'1960s': ['1960', '60s', '1960s']}

Query: Find me sad rock songs from the 1980s
Mapped Genres: {'rock'}
Mapped Moods: {'Mellow / Vibrant / Joyful'}
Mapped Decades: {'1980s': ['1980', '80s', '1980s']}

Query: Can you play some romantic classic jazz from the 20s?
Mapped Genres: {'jazz'}
Mapped Moods: {'Mellow / Vibrant / Joyful'}
Mapped Decades: {'1920s': ['1920', '20s', '1920s'], '2020s': ['2020', '20s', '2020s']}



In [1]:
from transformers import pipeline

# Load a pre-trained sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def analyze_sentiment(query):
    # Use the model to get sentiment prediction
    result = sentiment_analyzer(query)
    # The result includes label and score, e.g., {'label': 'POSITIVE', 'score': 0.999}
    # Convert label to a sentiment score
    score = result[0]['score']
    if result[0]['label'] == 'POSITIVE':
        return score  # Use positive scores directly
    else:
        return -score  # Convert negative sentiment to negative scores

# Example usage
query = "I love this happy song!"
print("Sentiment Score:", analyze_sentiment(query))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Sentiment Score: 0.9998853206634521


In [2]:
import pandas as pd
import re

# Enhanced mapping dictionary
genre_keywords = {
    'rock': ['rock', 'punk', 'grunge', 'alternative', 'indie rock', 'garage rock', 'psychedelic rock', 'classic rock', 'folk rock', 'hard rock', 'prog rock', 'post-rock'],
    'indie pop': ['pop', 'synthpop', 'indie pop', 'k-pop', 'pop rock', 'electropop', 'bubblegum pop', 'dance pop', 'pop punk'],
    'hip hop': ['hip hop', 'rap', 'trap', 'gangster rap', 'old school hip hop', 'underground hip hop', 'latin hip hop', 'jazz rap', 'desi hip hop'],
    'drum and bass': ['electronic', 'edm', 'house', 'techno', 'dubstep', 'trance', 'drum and bass', 'ambient', 'electro', 'industrial', 'techno'],
    'jazz funk': ['jazz', 'bebop', 'fusion', 'swing', 'smooth jazz', 'jazz funk', 'jazz fusion', 'acid jazz', 'jazz blues'],
    'classical': ['classical', 'opera', 'symphony', 'orchestral', 'baroque', 'chamber music', 'sonata', 'classical performance', 'early music'],
    'folk': ['folk', 'acoustic', 'singer-songwriter', 'folktronica', 'celtic', 'americana', 'bluegrass', 'folk rock'],
    'blues': ['blues', 'soul', 'gospel', 'r&b', 'soul blues', 'southern soul', 'blues rock', 'chicago blues', 'delta blues'],
    'reggae': ['reggae', 'dancehall', 'ska', 'dub', 'reggaeton', 'rocksteady', 'ragga', 'reggae fusion'],
    'country': ['country', 'country rock', 'alt-country', 'country blues', 'bluegrass', 'honky tonk', 'country pop', 'country soul'],
    'latin': ['latin', 'salsa', 'tango', 'reggaeton', 'latin pop', 'latin jazz', 'bossa nova', 'merengue', 'bachata', 'cumbia'],
    'world': ['world', 'afrobeat', 'fado', 'flamenco', 'k-pop', 'j-pop', 'regional mexican', 'samba', 'world fusion'],
    'electronic variations': ['ambient', 'synthwave', 'vaporwave', 'chillwave', 'downtempo', 'lo-fi', 'future bass', 'trip hop'],
    'death metal': ['metal', 'death metal', 'black metal', 'doom metal', 'thrash metal', 'folk metal', 'gothic metal', 'metalcore', 'viking metal'],
    'regional': ['french pop', 'italian pop', 'german pop', 'spanish pop', 'swedish pop', 'russian pop', 'polish pop', 'balkan', 'turkish pop'],
    'bengali': ['bengali', 'modern bollywood', 'hindi bollywood', 'bollywood']

}

mood_context_mapping = {
    'chill': ['Mellow / Vibrant / Joyful', 'Instrument-rich / Mellow / Danceable'],
    'vibrant': ['Varied Tempo / Vibrant / Mellow', 'Instrument-rich / Mellow / Danceable'],
    'joyful': ['Joyful / Danceable / Live', 'Mellow / Vibrant / Joyful'],
    'danceable': ['Joyful / Danceable / Live', 'Lyrical / Live / Danceable'],
    'lyrical': ['Lyrical / Live / Danceable'],
    'sad': ['Mellow / Vibrant / Joyful'],  # Assuming sad music might be more mellow and introspective
    'energetic': ['Joyful / Danceable / Live'],  # High-energy music for workouts or lively activities
    'relaxing': ['Instrument-rich / Mellow / Danceable', 'Mellow / Vibrant / Joyful'],  # Music for relaxation or stress relief
    'romantic': ['Mellow / Vibrant / Joyful'],  # Music that sets a romantic mood
    'intense': ['Varied Tempo / Vibrant / Mellow'],  # For thrilling or emotionally intense music
    'meditative': ['Instrument-rich / Mellow / Danceable']  # Music conducive to meditation or deep thought
}

# Define a function to generate keywords for decades
def generate_decade_keywords():
    temporal_keywords = {}
    for year in range(1920, 2030, 10):
        decade = year // 10 * 10
        decade_key = f"{decade}s"
        temporal_keywords[decade_key] = [
            str(decade),  # "1920"
            f"{str(decade)[2:]}s",  # "20s"
            f"{decade}s"  # "1920s"
        ]
    return temporal_keywords

# Generate the dictionary with temporal keywords
temporal_keywords = generate_decade_keywords()

def map_query_to_genres_moods_and_decades(query):
    query = query.lower()
    found_genres = set()
    found_moods = set()
    found_years = set()

    # Extract genres
    for genre, keywords in genre_keywords.items():
        if any(keyword in query for keyword in keywords):
            found_genres.add(genre)
            #print(f"Genre '{genre}' matched with keywords: {keywords}")

    # Extract moods
    for mood, descriptions in mood_context_mapping.items():
        if mood in query:
            for description in descriptions:
                found_moods.add(description)
            #print(f"Mood '{mood}' matched with descriptions: {descriptions}")

    # Extract decades and convert to years
    for decade, keywords in temporal_keywords.items():
        for keyword in keywords:
            if keyword in query:
                start_year = int(decade[:-1])  # Converts "2010s" to 2010
                found_years.update(range(start_year, start_year + 10))
                #print(f"Decade '{decade}' matched with keywords: {keywords}")

    return found_genres, found_moods, found_years


# Analyze sentiment based on the query
def analyze_sentiment(query):
    return 0.5 if "happy" in query else -0.5 if "sad" in query else 0

def recommend_songs(song_data, genres, moods, decades, sentiment_score):
    #print("Initial song count:", len(song_data))
    #print("Genres trying to match:", genres)

    # Debugging: Check what genres are available in the dataset
    #print("Unique genres in dataset:", song_data['genres'].str.split(' / ').explode().str.strip().unique())

    # Filter based on genres, if provided
    if genres:
        pattern = '|'.join([re.escape(genre) for genre in genres])  # Use regex escape to handle any special characters
        song_data = song_data[song_data['genres'].str.contains(pattern, case=False, na=False)]
        print("After genre filter:", len(song_data))
    else:
        print("No genre filtering applied.")

    # Filter based on moods, if provided
    if moods:
        song_data = song_data[song_data['Cluster_Description'].isin(moods)]
        print("After mood filter:", len(song_data))
    else:
        print("No mood filtering applied.")

    # Filter based on decades, if provided
    if decades:
        song_data = song_data[song_data['year'].isin(decades)]
        print("After decade filter:", len(song_data))
    else:
        print("No decade filtering applied.")

    # Check if there are any songs left to process
    if song_data.empty:
        print("No songs found after filtering.")
        return pd.DataFrame()

    # Apply sentiment adjustment to the 'popularity' score
    song_data['adjusted_score'] = song_data['popularity'] * (1 + sentiment_score)
    song_data.sort_values(by='adjusted_score', ascending=False, inplace=True)

    # Drop duplicates based on track name to ensure uniqueness in recommendations
    song_data = song_data.drop_duplicates(subset=['trackname', 'artistname'])

    # Return the top 3 results
    return song_data[['trackname', 'artistname', 'genres', 'year', 'adjusted_score']].head(10)

# Example usage
user_query = "give me energetic hip hop from 90s"
genres, moods, decades = map_query_to_genres_moods_and_decades(user_query)
sentiment_score = analyze_sentiment(user_query)
song_data = pd.read_csv('/content/drive/My Drive/DATASET/aggregated_data_full.csv')

recommendations = recommend_songs(song_data, genres, moods, decades, sentiment_score)
print("Recommended Songs:")
print(recommendations)





After genre filter: 294368
After mood filter: 202141
After decade filter: 61458
Recommended Songs:
                               trackname                artistname  \
654986                         No Scrubs                       tlc   
3656435              Killing In The Name  rage against the machine   
2944808  Killing Me Softly with His Song                    fugees   
2950980  Killing Me Softly With His Song                    fugees   
350109                         No scrubs                       tlc   
1141759              Killing In the Name  rage against the machine   
3145904              Killing in the Name  rage against the machine   
2483004              Killing in the name  rage against the machine   
2919070             Ambitionz Az A Ridah                      2pac   
466685              Ambitionz Az a Ridah                      2pac   

                                                    genres  year  \
654986   atl hip hop / dance pop / girl group / hip hop...  19

In [10]:
from transformers import pipeline
import pandas as pd
import re

# Load the sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def analyze_sentiment(query):
    # Analyze sentiment and convert to a score
    result = sentiment_analyzer(query)
    score = result[0]['score']
    return score if result[0]['label'] == 'POSITIVE' else -score

def generate_decade_keywords():
    # Generate keywords for each decade to filter by year
    temporal_keywords = {}
    for year in range(1920, 2030, 10):
        decade_key = f"{year // 10 * 10}s"
        temporal_keywords[decade_key] = [str(year // 10 * 10), f"{str(year // 10 * 10)[2:]}s", decade_key]
    return temporal_keywords

# Create a dictionary for mapping decades to keywords
temporal_keywords = generate_decade_keywords()

def map_query_to_genres_moods_and_decades(query):
    query = query.lower()
    found_genres, found_moods, found_years = set(), set(), set()

    # Extract genres from the query
    genre_keywords = {
    'rock': ['rock', 'punk', 'grunge', 'alternative', 'indie rock', 'garage rock', 'psychedelic rock', 'classic rock', 'folk rock', 'hard rock', 'prog rock', 'post-rock'],
    'indie pop': ['pop', 'synthpop', 'indie pop', 'k-pop', 'pop rock', 'electropop', 'bubblegum pop', 'dance pop', 'pop punk'],
    'hip hop': ['hip hop', 'rap', 'trap', 'gangster rap', 'old school hip hop', 'underground hip hop', 'latin hip hop', 'jazz rap', 'desi hip hop'],
    'electronic': ['electronic', 'edm', 'house', 'techno', 'dubstep', 'trance', 'drum and bass', 'ambient', 'electro', 'industrial', 'techno'],
    'jazz funk': ['jazz', 'bebop', 'fusion', 'swing', 'smooth jazz', 'jazz funk', 'jazz fusion', 'acid jazz', 'jazz blues'],
    'classical': ['classical', 'opera', 'symphony', 'orchestral', 'baroque', 'chamber music', 'sonata', 'classical performance', 'early music'],
    'folk': ['folk', 'acoustic', 'singer-songwriter', 'folktronica', 'celtic', 'americana', 'bluegrass', 'folk rock'],
    'blues': ['blues', 'soul', 'gospel', 'r&b', 'soul blues', 'southern soul', 'blues rock', 'chicago blues', 'delta blues'],
    'reggae': ['reggae', 'dancehall', 'ska', 'dub', 'reggaeton', 'rocksteady', 'ragga', 'reggae fusion'],
    'country': ['country', 'country rock', 'alt-country', 'country blues', 'bluegrass', 'honky tonk', 'country pop', 'country soul'],
    'latin': ['latin', 'salsa', 'tango', 'reggaeton', 'latin pop', 'latin jazz', 'bossa nova', 'merengue', 'bachata', 'cumbia'],
    'world': ['world', 'afrobeat', 'fado', 'flamenco', 'k-pop', 'j-pop', 'regional mexican', 'samba', 'world fusion'],
    'electronic variations': ['ambient', 'synthwave', 'vaporwave', 'chillwave', 'downtempo', 'lo-fi', 'future bass', 'trip hop'],
    'death metal': ['metal', 'death metal', 'black metal', 'doom metal', 'thrash metal', 'folk metal', 'gothic metal', 'metalcore', 'viking metal'],
    'regional': ['french pop', 'italian pop', 'german pop', 'spanish pop', 'swedish pop', 'russian pop', 'polish pop', 'balkan', 'turkish pop'],
    'bollywood': ['bengali', 'modern bollywood', 'hindi bollywood', 'bollywood']
    }

    # Check if any genre keywords are in the query
    for genre, keywords in genre_keywords.items():
        if any(keyword in query for keyword in keywords):
            found_genres.add(genre)

    # Example mood extraction logic
    mood_context_mapping = {
    'chill': ['Mellow / Vibrant / Joyful', 'Instrument-rich / Mellow / Danceable'],
    'vibrant': ['Varied Tempo / Vibrant / Mellow', 'Instrument-rich / Mellow / Danceable'],
    'joyful': ['Joyful / Danceable / Live', 'Mellow / Vibrant / Joyful'],
    'danceable': ['Joyful / Danceable / Live', 'Lyrical / Live / Danceable'],
    'lyrical': ['Lyrical / Live / Danceable'],
    'sad': ['Mellow / Vibrant / Joyful'],
    'energetic': ['Joyful / Danceable / Live'],
    'relaxing': ['Instrument-rich / Mellow / Danceable', 'Mellow / Vibrant / Joyful'],
    'romantic': ['Mellow / Vibrant / Joyful'],
    'intense': ['Varied Tempo / Vibrant / Mellow'],
    'meditative': ['Instrument-rich / Mellow / Danceable']
    }

    # Check if any mood descriptions are in the query
    for mood, descriptions in mood_context_mapping.items():
        if any(desc in query for desc in descriptions):
            found_moods.add(mood)

    # Extract decades and convert to years
    temporal_keywords = generate_decade_keywords()
    for decade, keywords in temporal_keywords.items():
        if any(keyword in query for keyword in keywords):
            start_year = int(decade[:-1])
            found_years.update(range(start_year, start_year + 10))

    return found_genres, found_moods, found_years

def recommend_songs(song_data, genres, moods, decades, sentiment_score):
    # Filter the song data based on genres, moods, and decades
    if genres:
        pattern = '|'.join([re.escape(genre) for genre in genres])
        song_data = song_data[song_data['genres'].str.contains(pattern, case=False, na=False)]

    if moods:
        song_data = song_data[song_data['moods'].isin(moods)]

    if decades:
        song_data = song_data[song_data['year'].isin(decades)]

    # Apply sentiment score to adjust the popularity score
    song_data['adjusted_score'] = song_data['popularity'] * (1 + sentiment_score)
    song_data.sort_values(by='adjusted_score', ascending=False, inplace=True)

    # Normalize and drop duplicates before final sorting
    song_data['trackname_normalized'] = song_data['trackname'].str.lower().str.strip()
    song_data['artistname_normalized'] = song_data['artistname'].str.lower().str.strip()
    song_data = song_data.drop_duplicates(subset=['trackname_normalized', 'artistname_normalized'])

    return song_data[['trackname', 'artistname', 'genres', 'year', 'adjusted_score']].head(10)

# Load song data from a CSV file
song_data = pd.read_csv('/content/drive/My Drive/DATASET/aggregated_data_full.csv')

# Example usage
user_query = "I'm looking for death metal tracks from 80s"
genres, moods, decades = map_query_to_genres_moods_and_decades(user_query)
sentiment_score = analyze_sentiment(user_query)
recommendations = recommend_songs(song_data, genres, moods, decades, sentiment_score)

print("Recommended Songs:")
print(recommendations)


Recommended Songs:
                        trackname     artistname  \
2261396             Raining Blood         slayer   
4354168           South Of Heaven         slayer   
832420             Angel Of Death         slayer   
4354041               Black Magic         slayer   
152457            The Toxic Waltz         exodus   
4350390                      Evil  mercyful fate   
2230859               Elimination       overkill   
3581717               Black Metal          venom   
1026959             Over The Wall      testament   
589543   Practice What You Preach      testament   

                                                    genres  year  \
2261396  alternative metal / death metal / hard rock / ...  1986   
4354168  alternative metal / death metal / hard rock / ...  1988   
832420   alternative metal / death metal / hard rock / ...  1986   
4354041  alternative metal / death metal / hard rock / ...  1983   
152457   death metal / groove metal / hard rock / metal...  1989   


In [33]:
import pandas as pd

# Load your song data
song_data = pd.read_csv('/content/drive/My Drive/DATASET/aggregated_data_full.csv')

# Filter for death metal songs
death_metal_songs = song_data[
    song_data['genres'].str.contains('death metal', case=False, na=False)
]

# Check the number of death metal songs and print some of them
print("Number of death metal songs:", len(death_metal_songs))
if not death_metal_songs.empty:
    print("Here are some death metal songs from the dataset:")
    print(death_metal_songs[['trackname', 'artistname', 'genres', 'year']].head(10))  # Displaying top 10 for brevity
else:
    print("No death metal songs were found.")


Number of death metal songs: 7467
Here are some death metal songs from the dataset:
              trackname         artistname  \
3960            Kaiowas          sepultura   
10347  Are You Dead Yet  children of bodom   
10529   South Of Heaven             slayer   
10530   South Of Heaven             slayer   
13534                99        the haunted   
13535                99        the haunted   
13562    Angel Of Death             slayer   
13563    Angel Of Death             slayer   
13564    Angel Of Death             slayer   
13572  Are You Dead Yet  children of bodom   

                                                  genres  year  
3960   alternative metal / brazilian groove metal / b...  1993  
10347  alternative metal / finnish death metal / finn...  2005  
10529  alternative metal / death metal / hard rock / ...  1988  
10530  alternative metal / death metal / hard rock / ...  1988  
13534  classic garage rock / death metal / gothenburg...  2004  
13535  classic gara

In [19]:
import pandas as pd

# Assuming song_data is already loaded, if not, load it:
song_data = pd.read_csv(f'/content/drive/My Drive/DATASET/aggregated_data_full.csv')

# Filter for hip hop songs from the year 2010
hip_hop_2010 = song_data[
    song_data['genres'].str.contains('death metal', case=False, na=False) &
    (song_data['year'] == 2010)
]

print("Number of hip hop songs from 2010:", len(hip_hop_2010))
if not hip_hop_2010.empty:
    print("Here are some hip hop songs from 2010:")
    print(hip_hop_2010[['trackname', 'artistname', 'genres', 'year']])
else:
    print("No hip hop songs from 2010 were found.")

Number of hip hop songs from 2010: 14194
Here are some hip hop songs from 2010:
                    trackname    artistname  \
3303     El Amor que Perdimos  prince royce   
3304     El Amor que Perdimos  prince royce   
4116        Don't Let Me Fall         b.o.b   
4160                Mr. Rager      kid cudi   
4161                Mr. Rager      kid cudi   
...                       ...           ...   
4403940     You’re Never Over        eminem   
4406760       Best I Ever Had         drake   
4406762       Best I Ever Had         drake   
4407748       Pretty Boy Swag    soulja boy   
4407989           Space Bound        eminem   

                                                    genres  year  
3303     bachata / latin / latin hip hop / latin pop / ...  2010  
3304     bachata / latin / latin hip hop / latin pop / ...  2010  
4116     atl hip hop / dance pop / hip hop / pop / pop ...  2010  
4160                hip hop / ohio hip hop / pop rap / rap  2010  
4161                

In [2]:
def recommend_songs(song_data, genres, moods, decades, sentiment_score):
    # Filter based on genres, moods, and decades
    filtered_songs = song_data[
        song_data['genres'].apply(lambda x: any(genre in x for genre in genres)) &
        song_data['Cluster_Description'].isin(moods) &
        song_data['year'].apply(lambda y: any(decade.startswith(str(y)[:3]) for decade in decades))
    ]

    # Assume 'popularity' is your base score. Adjust scores based on sentiment
    filtered_songs['adjusted_score'] = filtered_songs['popularity'] * (1 + sentiment_score)

    # Drop duplicates based on track name and artist, keeping the entry with the highest adjusted score
    filtered_songs = filtered_songs.sort_values(by='adjusted_score', ascending=False)
    unique_songs = filtered_songs.drop_duplicates(subset=['trackname', 'artistname'])

    # Return top 3 recommendations
    return unique_songs[['trackname', 'artistname', 'genres', 'year']].head(3)

# Example usage
user_query = "I want to listen to some chill jazz music from the 60s that makes me happy"
genres, moods, decades = map_query_to_genres_moods_and_decades(user_query)
sentiment_score = analyze_sentiment(user_query)
song_data = pd.read_csv(f'/content/drive/My Drive/DATASET/aggregated_data_full.csv')  # Load your song data

recommendations = recommend_songs(song_data, genres, moods, decades, sentiment_score)

print("Recommended Songs:")
print(recommendations)
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Function to load the model and tokenizer
def load_model():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
    return tokenizer, model

# Load tokenizer and model
tokenizer, model = load_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import torch

# Load a pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Create a pipeline for named entity recognition
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

test_query = "I want to listen to some jazz and blues."
ner_results = ner_pipeline(test_query)
print(ner_results)

Found Genres: ['jazz', 'blues']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Example dataset
data = {
    'text': ['play some rock music', 'I need songs for a workout', 'find relaxing jazz music'],
    'intent': ['genre', 'activity', 'mood']
}

# Convert text data into a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data['text'])

# Train the model to classify intents
model = MultinomialNB()
model.fit(X_train, data['intent'])

# Creating a pipeline that processes text and predicts intent
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.fit(data['text'], data['intent'])

# Predicting the intent of a new query
query = "I want music for studying"
predicted_intent = pipe.predict([query])[0]
print("Predicted Intent:", predicted_intent)

Predicted Intent: activity



Fully Aggregated Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18136453 entries, 0 to 18136452
Data columns (total 9 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   user_id              object
 1   trackname            object
 2   artistname           object
 3   playlistname         object
 4   cluster              int64 
 5   Cluster_Description  object
 6   year                 int64 
 7   popularity           int64 
 8   play_count           int64 
dtypes: int64(4), object(5)
memory usage: 1.2+ GB
None

Sample records from the fully aggregated dataset:
                                   user_id          trackname  \
8277259   7292145b6a7c4d99712fe606679fdee5          Heartless   
16089206  e30801ce969fa96b8995f5f6c181b475  The Perfect Space   
2134290   1e413aa46fc2f7e4530c84013d64d942              Bones   
15143342  d49c0fdadc701f66d10aec118604f4b7           Hey Jude   
6637393   5bec076865f710a6314a93765c324633     Open Your

In [11]:
from transformers import pipeline

# Specify the model and revision explicitly for the sentiment analysis task
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pipeline_name = "text-classification"
nlp_model = pipeline(pipeline_name, model=model_name)

def predict_query_details(query):
    """
    Predicts details from the query using the specified NLP model.
    """
    result = nlp_model(query)
    sentiment = result[0]['label']
    return sentiment


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [25]:
cluster_descriptions = agg_data['Cluster_Description'].sample(10)
print(cluster_descriptions)


10524782         Mellow / Vibrant / Joyful / Classical / Jazz
5976109               Joyful / Danceable / Live / Pop / Dance
10049839         Mellow / Vibrant / Joyful / Classical / Jazz
9627453     Varied Tempo / Vibrant / Mellow / Indie / Alte...
17195767    Varied Tempo / Vibrant / Mellow / Indie / Alte...
8559406               Joyful / Danceable / Live / Pop / Dance
9854105          Mellow / Vibrant / Joyful / Classical / Jazz
16672653              Joyful / Danceable / Live / Pop / Dance
8326425     Varied Tempo / Vibrant / Mellow / Indie / Alte...
13607873              Joyful / Danceable / Live / Pop / Dance
Name: Cluster_Description, dtype: object


In [42]:
# Maps for cluster descriptions and genre tags
base_cluster_descriptions = {
    0: "Varied Tempo / Vibrant / Mellow",
    1: "Joyful / Danceable / Live",
    2: "Lyrical / Live / Danceable",
    3: "Mellow / Vibrant / Joyful",
    4: "Instrument-rich / Mellow / Danceable",
}

genre_tags = {
    0: ['Indie', 'Alternative'],
    1: ['Pop', 'Dance'],
    2: ['Hip-Hop', 'Rap'],
    3: ['Classical', 'Jazz'],
    4: ['Rock', 'Metal']
}

genre_synonyms = {
    'indie': ['Indie', 'Alternative'],
    'alternative': ['Indie', 'Alternative'],
    'pop': ['Pop'],
    'dance': ['Dance'],
    'hip hop': ['Hip-Hop'],
    'rap': ['Rap'],
    'classical': ['Classical'],
    'jazz': ['Jazz'],
    'rock': ['Rock'],
    'metal': ['Metal'],
    'funk': ['Funk', 'Soul'],
    'soul': ['Soul', 'R&B'],
    'electronic': ['Electronic', 'Dance'],
    'folk': ['Folk', 'Acoustic'],
    'acoustic': ['Acoustic'],
    'blues': ['Blues'],
    'r&b': ['R&B'],
    'reggae': ['Reggae'],
    'country': ['Country']
}

# Map feature names to intuitive descriptions
feature_names_map = {
    'tempo': 'Varied Tempo', 'energy': 'Vibrant', 'valence': 'Joyful',
    'danceability': 'Danceable', 'speechiness': 'Lyrical', 'acousticness': 'Mellow',
    'instrumentalness': 'Instrument-rich', 'liveness': 'Live', 'loudness': 'Loud'
}

In [45]:
import pandas as pd
import numpy as np
import re

def predict_query_details(query):
    if 'sad' in query.lower():
        return 'NEGATIVE'
    else:
        return 'POSITIVE'

def extract_genre(query):
    found_genres = set()
    query_lower = query.lower()
    for key, values in genre_synonyms.items():
        if re.search(r'\b' + key + r'\b', query_lower):
            found_genres.update(values)
    return list(found_genres) if found_genres else None

def map_genres_to_clusters(genres):
    clusters = set()
    for genre in genres:
        for cluster, tags in genre_tags.items():
            if genre in tags:
                clusters.add(cluster)
    return list(clusters)

def extract_year(query):
    match = re.search(r'\b(19|20)\d{2}\b', query)
    if match:
        return int(match.group(0))
    match = re.search(r'\b(80s|90s|2000s|70s|60s)\b', query)
    if match:
        decade = match.group(0).rstrip('s')
        return 2000 if '2000' in decade else int('19' + decade)
    return None

def recommend_songs(query, agg_data, top_n=10):
    sentiment = predict_query_details(query)
    year = extract_year(query)
    genres = extract_genre(query)
    clusters = map_genres_to_clusters(genres) if genres else []

    print(f"Predicted sentiment: {sentiment}, Target year: {year}, Genres: {genres}")

    filtered_songs = agg_data.copy()

    # Filter by year range
    if year:
        decade_start = (year // 10) * 10
        decade_end = decade_start + 9
        filtered_songs = filtered_songs[(filtered_songs['year'] >= decade_start) & (filtered_songs['year'] <= decade_end)]

    # Filter by sentiment/popularity
    if sentiment == 'NEGATIVE':
        filtered_songs = filtered_songs[filtered_songs['popularity'] <= 50]

    # Filter by genre using a regex pattern to match any of the genres
    if genres:
        genre_regex = '|'.join(genres)  # Create a regex pattern from genres
        filtered_songs = filtered_songs[filtered_songs['Cluster_Description'].str.contains(genre_regex, case=False, na=False)]

    # Drop duplicates and select top N
    filtered_songs = filtered_songs.drop_duplicates(subset=['trackname'])
    return filtered_songs[['trackname', 'artistname', 'year', 'popularity']].head(top_n)


# Example usage
dataset_path = '/content/drive/My Drive/DATASET'
agg_data = pd.read_csv(f'{dataset_path}/aggregated_data_full.csv')
query = "I want to hear some sad metal from the 80s"
recommendations = recommend_songs(query, agg_data)
print("Recommendations based on your query:")
print(recommendations)


Predicted sentiment: NEGATIVE, Target year: 1980, Genres: ['Metal']
Recommendations based on your query:
                   trackname        artistname  year  popularity
89                   Friends        Ed Sheeran  1984          22
421                  Call Me         Shinedown  1986          33
573                      One             Creed  1985          34
760                 Cavatina    Sofia Källgren  1984          46
800            Eleanor Rigby   Peter Johansson  1985          32
1062              Moon River   Peter Johansson  1987          39
1231            Silent Night      Solid Gospel  1980          22
1332         White Christmas  Christer Sjögren  1984          26
1371         'Round Midnight        Lee Konitz  1987          33
1422  All The Things You Are     Serge Chaloff  1983          31


In [38]:
# Check entries from the 1980s
eighties_songs = agg_data[(agg_data['year'] >= 1980) & (agg_data['year'] < 1990)]
print("Number of songs from the 1980s:", len(eighties_songs))
print(eighties_songs[['trackname', 'artistname', 'year', 'popularity']].head(20))

# Check for specific genres
#indie_songs = agg_data[agg_data['Cluster_Description'].str.contains('Indie', case=False, na=False)]
#print("Number of indie songs:", len(indie_songs))
#print(indie_songs[['trackname', 'artistname', 'year', 'popularity']].head())


Number of songs from the 1980s: 2009146
          trackname     artistname  year  popularity
7    After Midnight      blink-182  1987          26
36              Boy     Little Mix  1986          39
81          Friends     Ed Sheeran  1982          29
82          Friends     Ed Sheeran  1982          33
83          Friends     Ed Sheeran  1989          35
88          Friends     Ed Sheeran  1983          26
89          Friends     Ed Sheeran  1984          22
96       Girlfriend  Avril Lavigne  1986          32
103           Girls       The 1975  1986          58
116       Good Life    OneRepublic  1989          60
127      High Hopes      The Vamps  1988          35
133       I Got You      Cimorelli  1980          50
134       I Got You      Cimorelli  1988          35
147      I Miss You      blink-182  1984          39
154     Imagination   Shawn Mendes  1983          40
155     Imagination   Shawn Mendes  1984          32
156     Imagination   Shawn Mendes  1988          43
166   

In [46]:
import pandas as pd

# Assuming dataset_path is defined and correct
dataset_path = '/content/drive/My Drive/DATASET'
agg_data = pd.read_csv(f'{dataset_path}/aggregated_data_full.csv')
genres_to_check = ['Metal', 'Pop', 'Jazz', 'Classical', 'Hip-Hop']
def check_genre_samples(data, genre_tags, num_samples=5):
    for genre in genre_tags:
        # Assuming genre tags directly correspond to descriptions or part of them
        genre_pattern = r'\b{}\b'.format(genre)
        filtered_data = data[data['Cluster_Description'].str.contains(genre_pattern, case=False, na=False)]
        print(f"Samples for genre {genre}:")
        print(filtered_data[['trackname', 'artistname', 'year', 'popularity']].head(num_samples))
        print("\n")

# Execute the check
check_genre_samples(agg_data, genres_to_check)


Samples for genre Metal:
          trackname    artistname  year  popularity
14   After Midnight     blink-182  2006          44
73       First Date     blink-182  2018          69
89          Friends    Ed Sheeran  1984          22
168     Imagination  Shawn Mendes  1953           9
169     Imagination  Shawn Mendes  1953          23


Samples for genre Pop:
         trackname     artistname  year  popularity
2               18  Jannik Brunke  2013          70
3               18  Jannik Brunke  2019          65
8   After Midnight      blink-182  1970          35
9   After Midnight      blink-182  1970          45
10  After Midnight      blink-182  1992          39


Samples for genre Jazz:
         trackname  artistname  year  popularity
12  After Midnight   blink-182  1972          64
13  After Midnight   blink-182  1997          45
37             Boy  Little Mix  1954           2
38             Boy  Little Mix  2017          65
50         Brother  Gerard Way  1992          41


Samp

In [47]:
import pandas as pd

# Assuming dataset_path is defined and correct
dataset_path = '/content/drive/My Drive/DATASET'
agg_data = pd.read_csv(f'{dataset_path}/aggregated_data_full.csv')
# Filter the dataset for entries that include "After Midnight"
after_midnight_entries = agg_data[agg_data['trackname'].str.contains('After Midnight', case=False, na=False)]

# Display all relevant details of these entries
print(after_midnight_entries[['trackname', 'artistname', 'year', 'popularity', 'Cluster_Description']])


               trackname artistname  year  popularity  \
7         After Midnight  blink-182  1987          26   
8         After Midnight  blink-182  1970          35   
9         After Midnight  blink-182  1970          45   
10        After Midnight  blink-182  1992          39   
11        After Midnight  blink-182  2011          52   
...                  ...        ...   ...         ...   
18090999  After Midnight  blink-182  1992          39   
18091000  After Midnight  blink-182  2011          52   
18091001  After Midnight  blink-182  1972          64   
18091002  After Midnight  blink-182  1997          45   
18091003  After Midnight  blink-182  2006          44   

                                        Cluster_Description  
7         Varied Tempo / Vibrant / Mellow / Indie / Alte...  
8                   Joyful / Danceable / Live / Pop / Dance  
9                   Joyful / Danceable / Live / Pop / Dance  
10                  Joyful / Danceable / Live / Pop / Dance  
11   

In [51]:
import pandas as pd

# Assuming dataset_path is defined and correct
dataset_path = '/content/drive/My Drive/DATASET'
genre_csv = pd.read_csv(f'{dataset_path}/data_w_genres.csv')
label_data = pd.read_csv(f'{dataset_path}/data_final_with_descriptive_labels.csv')

# Display the structure of the loaded data to ensure it's loaded correctly
print("\nFully Aggregated Data Information:")
print(genre_csv.info())

# Display a few random samples to get a sense of the data
print("\nSample records from the fully aggregated dataset:")
print(genre_csv.sample(10))

# Display the structure of the loaded data to ensure it's loaded correctly
print("\nFully Aggregated Data Information:")
print(label_data.info())

# Display a few random samples to get a sense of the data
print("\nSample records from the fully aggregated dataset:")
print(label_data.sample(10))




Fully Aggregated Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28680 entries, 0 to 28679
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            28680 non-null  object 
 1   artists           28680 non-null  object 
 2   acousticness      28680 non-null  float64
 3   danceability      28680 non-null  float64
 4   duration_ms       28680 non-null  float64
 5   energy            28680 non-null  float64
 6   instrumentalness  28680 non-null  float64
 7   liveness          28680 non-null  float64
 8   loudness          28680 non-null  float64
 9   speechiness       28680 non-null  float64
 10  tempo             28680 non-null  float64
 11  valence           28680 non-null  float64
 12  popularity        28680 non-null  float64
 13  key               28680 non-null  int64  
 14  mode              28680 non-null  int64  
 15  count             28680 non-null  int64  
dtypes: f

In [54]:
# Standardizing and preparing the artists column for a consistent merge
genre_csv['artists'] = genre_csv['artists'].str.lower().str.strip()
label_data['artists'] = label_data['artists'].str.lower().str.strip()

# Merging the datasets on the 'artists' column
merged_data = pd.merge(
    label_data[['artists', 'popularity', 'year', 'cluster', 'Cluster_Description']],  # from label_data
    genre_csv[['artists', 'genres']],  # from genre_csv
    on='artists',
    how='left'
)
# Displaying a sample of the merged data to review the results
print(merged_data.sample(10))
print(merged_data.info())



                                        artists  popularity  year  cluster  \
63560          ['cannonball adderley', 'sweet']          22  1962        3   
24054    ['giuseppe verdi', 'arturo toscanini']           1  1946        3   
94520   ['ludwig van beethoven', 'isaac stern']           0  1949        3   
14016                       ['alice in chains']          71  1992        0   
166092    ['grupo cañaveral de humberto pabón']          42  1997        1   
19833       ['jimmy rushing and his orchestra']           0  1924        3   
99086                ['michael martin murphey']          29  1972        3   
94003         ['rajkumari', 'hariprasanna das']           0  1946        3   
59459        ['isaac albéniz', 'claudio arrau']           0  1941        4   
125502                     ['giorgos kavouras']           0  1940        4   

                         Cluster_Description genres  
63560              Mellow / Vibrant / Joyful    NaN  
24054              Mellow / Vibra

In [55]:
# Example: Check for different formatting in artist names
print("Sample from genre_csv:", genre_csv['artists'].sample(5))
print("Sample from label_data:", label_data['artists'].sample(5))

Sample from genre_csv: 8661           funkadelic
27892          young nudy
5617          daniel hope
18361    norman greenbaum
4745              clannad
Name: artists, dtype: object
Sample from label_data: 80781                   ['herb alpert & the tijuana brass']
19863                                   ['ignacio corsini']
190                                      ['adolphe bérard']
48665                                 ['vicente fernández']
126174    ['wolfgang amadeus mozart', 'george szell', 'c...
Name: artists, dtype: object


In [56]:
import ast  # For safely evaluating strings containing Python expressions

def clean_artists_names(x):
    if x.startswith('['):
        # Convert string representation of list to actual list
        x = ast.literal_eval(x)
        # If it's a list, join with comma or return the first artist, choose based on your need
        return ', '.join(x)  # or return x[0] if you want just the first artist
    return x.strip()

# Clean the artists columns
label_data['artists'] = label_data['artists'].apply(clean_artists_names)
genre_csv['artists'] = genre_csv['artists'].str.strip()
merged_data = pd.merge(
    label_data[['artists', 'popularity', 'year', 'cluster', 'Cluster_Description']],
    genre_csv[['artists', 'genres']],
    on='artists',
    how='left'
)

# Check the merged data to ensure genres are now matching correctly
print(merged_data.sample(10))
print(merged_data.info())
# Check how many entries now have genre data
print("Entries with genre information:", merged_data['genres'].notna().sum())


                                                  artists  popularity  year  \
63624                             stan getz, charlie byrd          21  1962   
17023                                            kid rock          74  2007   
77366                                   эрих мария ремарк           0  1945   
8735                                   the rolling stones          54  1965   
119465                                           ice cube          37  1993   
87233                                            megadeth          45  1995   
78296   gustav mahler, bruno walter, new york philharm...           0  1949   
160790                                       leon russell          20  1971   
73726                                        george jones          54  2013   
126408                                       hemant kumar           0  1948   

        cluster                   Cluster_Description  \
63624         3             Mellow / Vibrant / Joyful   
17023         1 

In [62]:
# Count entries per artist in the genre_csv dataset
genre_artist_counts = genre_csv.groupby('artists').size().reset_index(name='genre_entry_count')

# Count entries per artist in the label_data dataset
label_artist_counts = label_data.groupby('artists').size().reset_index(name='label_entry_count')

# Merging entry counts into the merged dataset
merged_data_with_counts = pd.merge(
    merged_data,
    genre_artist_counts,
    on='artists',
    how='left'
)

merged_data_with_counts = pd.merge(
    merged_data_with_counts,
    label_artist_counts,
    on='artists',
    how='left'
)

# Check the count distribution in the merged dataset
print(merged_data_with_counts.sample(10))

# Generate summary statistics for these counts
print("Summary stats for genre_entry_count:")
print(merged_data_with_counts['genre_entry_count'].describe())

print("Summary stats for label_entry_count:")
print(merged_data_with_counts['label_entry_count'].describe())

# Additional checks
# Total number of unique artists in each original dataset
unique_artists_genre = genre_artist_counts['artists'].nunique()
unique_artists_label = label_artist_counts['artists'].nunique()

# Check how many artists are in the merged dataset
unique_artists_merged = merged_data_with_counts['artists'].nunique()

print(f"Unique artists in genre_csv: {unique_artists_genre}")
print(f"Unique artists in label_data: {unique_artists_label}")
print(f"Unique artists in merged dataset: {unique_artists_merged}")




                              artists  popularity  year  cluster  \
141279                             周璇           2  1940        4   
168294                         barney          44  2009        1   
119019                       obituary          37  1990        0   
114174                           love          15  1966        1   
102065                        erasure          28  1986        1   
162654  grateful dead, tom constanten          20  1980        3   
158299        philly joe jones sextet           4  1958        3   
76438                  h.p. lovecraft           0  1936        2   
65862                           slade          54  1974        0   
116950                          queen          25  1980        0   

                         Cluster_Description  \
141279  Instrument-rich / Mellow / Danceable   
168294             Joyful / Danceable / Live   
119019       Varied Tempo / Vibrant / Mellow   
114174             Joyful / Danceable / Live   
102065     

In [63]:
import numpy as np

# Set NaN values to 'Unknown'
merged_data_with_counts['genres'] = merged_data_with_counts['genres'].apply(lambda x: 'Unknown' if x is np.nan or x == '[]' else x)

# Normalize empty lists to 'Unknown'
merged_data_with_counts['genres'] = merged_data_with_counts['genres'].apply(lambda x: 'Unknown' if not x else x)

def normalize_genres(genre_list):
    if genre_list == 'Unknown':
        return 'Unknown'
    # Assuming the genre_list is a string representation of a list
    genre_list = ast.literal_eval(genre_list) if isinstance(genre_list, str) else genre_list
    return ' / '.join(sorted(set(genre_list)))  # Sort and remove duplicates

merged_data_with_counts['genres'] = merged_data_with_counts['genres'].apply(normalize_genres)

merged_data_with_counts.drop(columns=['genre_entry_count', 'label_entry_count'], inplace=True)

# Display a sample of the dataset
print(merged_data_with_counts.sample(10))

# Print information about the dataset
print(merged_data_with_counts.info())

# Check for any remaining missing data
missing_data = merged_data_with_counts.isnull().sum()
print("Missing data in each column:\n", missing_data)


                                                  artists  popularity  year  \
134632                              maze, frankie beverly          27  1989   
143706                                          peggy lee          14  1958   
15357                                          black star          58  1998   
112274                                      mohammed rafi          13  1957   
42299                   felix mendelssohn, william kapell           0  1945   
25642   wolfgang amadeus mozart, glyndebourne festival...          15  1955   
37196                                       ariana grande          59  2013   
147378                                      gino vannelli          27  1978   
166901                                       sister nancy          42  2002   
42171                                       alberto gomez           0  1944   

        cluster                   Cluster_Description  \
134632        1             Joyful / Danceable / Live   
143706        3 

In [64]:
pip install fuzzywuzzy
